## Python Ni DAQ

#### Reading Single Data

In [ ]:
import nidaqmx

with nidaqmx.Task() as task:
    task.ai_channels.add_ai_voltage_chan("Dev1/ai1")
    single_sample = task.read()
    print(f"Single Sample: {single_sample}")


#### Continuous Data Reading

In [ ]:
import nidaqmx
from nidaqmx.constants import AcquisitionType

with nidaqmx.Task() as task:
    task.ai_channels.add_ai_voltage_chan("Dev1/ai1")
    task.timing.cfg_samp_clk_timing(rate=1000, sample_mode=AcquisitionType.CONTINUOUS)

    print("Reading samples...")
    try:
        while True:
            data = task.read(number_of_samples_per_channel=10)
            print(data)
    except KeyboardInterrupt:
        print("Continuous reading stopped by user.")


#### Writing Single Data

In [ ]:
import nidaqmx

with nidaqmx.Task() as task:
    task.ao_channels.add_ao_voltage_chan("Dev1/ao0")
    task.write(5.0)  # Writing a single voltage value
    print("Wrote 5V to ao0")


#### Writing Continuous Data

In [ ]:
import nidaqmx
from nidaqmx.constants import AcquisitionType
import numpy as np

with nidaqmx.Task() as task:
    task.ao_channels.add_ao_voltage_chan("Dev1/ao0")
    task.timing.cfg_samp_clk_timing(rate=1000, sample_mode=AcquisitionType.CONTINUOUS)

    samples_to_write = np.linspace(0, 5, 100)
    task.write(samples_to_write, auto_start=True)

    input("Generating voltage ramp. Press Enter to stop...")


#### Mini Project: Real-Time Data Acquisition and Data Logging

In [ ]:
import nidaqmx
from nidaqmx.constants import AcquisitionType
import datetime

# Configuration
sampling_rate = 1000
channel = "Dev1/ai1"

# Setting up the DAQ Task
with nidaqmx.Task() as task:
    task.ai_channels.add_ai_voltage_chan(channel)
    task.timing.cfg_samp_clk_timing(rate=sampling_rate, sample_mode=AcquisitionType.CONTINUOUS)

    # Data logging setup
    with open("data_log.csv", "a") as file:
        print("Logging data... (Ctrl+C to stop)")
        try:
            while True:
                data = task.read(number_of_samples_per_channel=10)
                timestamp = datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S")
                for value in data:
                    file.write(f"{timestamp},{value}\n")
        except KeyboardInterrupt:
            print("Data logging stopped by user.")


#### Mini Project 2: Power Supply for a 4-Point Measurement System

In [ ]:
import nidaqmx
import time
import csv

# Constants
VOLTAGE_TO_CURRENT_SCALE = 5  # 1V = 5A
MAX_CURRENT = 0.5  # 500 mA
STEP_SIZE = 0.01  # Current step size (10 mA)
MEASUREMENT_INTERVAL = 1  # Time in seconds between measurements

# File for logging data
output_file = "measurement_log.csv"

# Function to write current
def write_current(desired_current):
    with nidaqmx.Task() as write_task:
        write_task.ao_channels.add_ao_voltage_chan("Dev1/ao0")
        scaled_voltage = desired_current / VOLTAGE_TO_CURRENT_SCALE
        write_task.write(scaled_voltage)
        print(f"Writing {desired_current}A to the system (Scaled Voltage: {scaled_voltage}V)")

# Function to read voltage
def read_voltage():
    with nidaqmx.Task() as read_task:
        read_task.ai_channels.add_ai_voltage_chan("Dev1/ai1")
        scaled_voltage = read_task.read()
        actual_voltage = scaled_voltage * VOLTAGE_TO_CURRENT_SCALE
        return actual_voltage

# Open the file for logging
with open(output_file, 'w', newline='') as file:
    writer = csv.writer(file)
    writer.writerow(['Current (A)', 'Voltage (V)', 'Resistance (Ohms)'])

    # Incrementally increase current and measure voltage
    current = 0
    while current <= MAX_CURRENT:
        write_current(current)
        time.sleep(MEASUREMENT_INTERVAL)  # Wait for the system to stabilize

        voltage = read_voltage()
        resistance = voltage / current if current != 0 else float('inf')

        # Log the data
        writer.writerow([current, voltage, resistance])
        print(f"Current: {current} A, Voltage: {voltage} V, Resistance: {resistance} Ohms")

        current += STEP_SIZE

print("Measurement completed. Data logged to", output_file)


#### Main Project: 4-Point Measurement System in a GUI

In [ ]:
import tkinter as tk
from tkinter import ttk
import nidaqmx
import csv
import threading

class MeasurementApp:
    def __init__(self, root):
        self.root = root
        self.measurement_running = False
        root.title("Resistivity Measurement System")

        # Measurement Parameters
        self.max_current = tk.DoubleVar(value=0.5)
        self.ao_channel = tk.StringVar(value="Dev1/ao0")
        self.ai_channel = tk.StringVar(value="Dev1/ai1")
        self.area = tk.DoubleVar(value=1.0)
        self.length = tk.DoubleVar(value=1.0)

        self.setup_ui()

    def setup_ui(self):
        tk.Label(root, text="Max Current (A):").grid(row=0, column=0)
        tk.Entry(root, textvariable=self.max_current).grid(row=0, column=1)

        tk.Label(root, text="AO Channel:").grid(row=1, column=0)
        tk.Entry(root, textvariable=self.ao_channel).grid(row=1, column=1)

        tk.Label(root, text="AI Channel:").grid(row=2, column=0)
        tk.Entry(root, textvariable=self.ai_channel).grid(row=2, column=1)

        tk.Label(root, text="Area (m^2):").grid(row=3, column=0)
        tk.Entry(root, textvariable=self.area).grid(row=3, column=1)

        tk.Label(root, text="Length (m):").grid(row=4, column=0)
        tk.Entry(root, textvariable=self.length).grid(row=4, column=1)

        tk.Button(root, text="Start", command=self.start_measurement).grid(row=5, column=0)
        tk.Button(root, text="Stop", command=self.stop_measurement).grid(row=5, column=1)

        self.status_label = tk.Label(root, text="Status: Idle")
        self.status_label.grid(row=6, column=0, columnspan=2)

        self.resistivity_label = tk.Label(root, text="Resistivity: Not calculated yet")
        self.resistivity_label.grid(row=7, column=0, columnspan=2)

    def start_measurement(self):
        if not self.measurement_running:
            self.measurement_running = True
            self.status_label.config(text="Status: Running")
            measurement_thread = threading.Thread(target=self.perform_measurement)
            measurement_thread.start()

    def stop_measurement(self):
        self.measurement_running = False
        self.status_label.config(text="Status: Stopped")

    def perform_measurement(self):
        with nidaqmx.Task() as write_task, nidaqmx.Task() as read_task, open("measurement_log.csv", 'w', 
                                                                             newline='') as file:
            writer = csv.writer(file)
            writer.writerow(['Current (A)', 'Voltage (V)', 'Resistance (Ohms)', 'Resistivity (Ohm*m)'])

            write_task.ao_channels.add_ao_voltage_chan(self.ao_channel.get())
            read_task.ai_channels.add_ai_voltage_chan(self.ai_channel.get())

            current = 0
            step_size = 0.01  # Increment current by 10 mA
            while current <= self.max_current.get() and self.measurement_running:
                scaled_voltage = current / 5  # 1V = 5A
                write_task.write(scaled_voltage)
                
                read_voltage = read_task.read() * 5  # Scale back the voltage
                resistance = read_voltage / current if current != 0 else float('inf')
                resistivity = resistance * (self.area.get() / self.length.get())

                writer.writerow([current, read_voltage, resistance, resistivity])
                self.resistivity_label.config(text=f"Resistivity: {resistivity} Ohm*m")
                current += step_size

            self.stop_measurement()

root = tk.Tk()
app = MeasurementApp(root)
root.mainloop()
